In [1]:
import boto3
import pysam
import os
import pandas as pd
import tempfile
import ast
import multiprocessing as mp
from multiprocessing import Pool
import numpy as np
cpus = int(np.floor((mp.cpu_count()-1) /2))
print(cpus)

35


## Setup

In [2]:
os.chdir('/data/jake/sv-gmm/python')
sv_tbl='../data/sv-modes.tsv'
dirout='../data/variants'
tmp='../tmp'
idx_1kg_phase3='../data/20130502.phase3.low_coverage.alignment.index' # from: aws s3 ls s3://1000genomes/alignment_indices/
if not os.path.exists(dirout):
    os.mkdir(dirout)

In [3]:
df = pd.read_csv(sv_tbl,sep='\t')
df.head()

,SV ID,# modes predicted,chr,start,stop,allele frequency,length,Total # samples,# reference samples,Mode 1,Mode 2,Mode 3
0,UW_VH_9038,1,19,54887338,54888354,0.014,1016,51,0,"['HG01125', 'HG01896', 'HG01956', 'HG02009', '...",NaN,NaN
1,SI_BD_10797,1,10,86801825,86802449,0.033,624,283,139,"['HG00154', 'HG00159', 'HG00250', 'HG00264', '...",NaN,NaN
2,UW_VH_19141,1,3,177294474,177297489,0.032,3015,133,1,"['HG00180', 'HG00185', 'HG00189', 'HG00237', '...",NaN,NaN
3,DEL_pindel_47187,2,18,45379612,45379612,0.550,195,847,66,"['HG00114', 'HG00132', 'HG00142', 'HG00150', '...","['HG00108', 'HG00111', 'HG00121', 'HG00125', '...",NaN
4,DEL_pindel_24042,2,7,136996507,136996739,0.071,232,184,48,"['HG01241', 'HG01259', 'HG01488', 'HG01890', '...","['HG01125', 'HG01392', 'HG01403', 'HG01556', '...",NaN


In [4]:
df.columns = ['id', 'n_modes', 'chr', 'start', 'stop', 'allele_freq', 'length', 'n_samples', 'n_ref_samples', 'mode_1', 'mode_2', 'mode_3']
df.head()

,id,n_modes,chr,start,stop,allele_freq,length,n_samples,n_ref_samples,mode_1,mode_2,mode_3
0,UW_VH_9038,1,19,54887338,54888354,0.014,1016,51,0,"['HG01125', 'HG01896', 'HG01956', 'HG02009', '...",NaN,NaN
1,SI_BD_10797,1,10,86801825,86802449,0.033,624,283,139,"['HG00154', 'HG00159', 'HG00250', 'HG00264', '...",NaN,NaN
2,UW_VH_19141,1,3,177294474,177297489,0.032,3015,133,1,"['HG00180', 'HG00185', 'HG00189', 'HG00237', '...",NaN,NaN
3,DEL_pindel_47187,2,18,45379612,45379612,0.550,195,847,66,"['HG00114', 'HG00132', 'HG00142', 'HG00150', '...","['HG00108', 'HG00111', 'HG00121', 'HG00125', '...",NaN
4,DEL_pindel_24042,2,7,136996507,136996739,0.071,232,184,48,"['HG01241', 'HG01259', 'HG01488', 'HG01890', '...","['HG01125', 'HG01392', 'HG01403', 'HG01556', '...",NaN


In [5]:
# mode columns were parsed as strings, not lists
df['mode_1'] = df['mode_1'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else x)
df['mode_2'] = df['mode_2'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else x)
df['mode_3'] = df['mode_3'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else x)
df.head()

,id,n_modes,chr,start,stop,allele_freq,length,n_samples,n_ref_samples,mode_1,mode_2,mode_3
0,UW_VH_9038,1,19,54887338,54888354,0.014,1016,51,0,"[HG01125, HG01896, HG01956, HG02009, HG02108, ...",NaN,NaN
1,SI_BD_10797,1,10,86801825,86802449,0.033,624,283,139,"[HG00154, HG00159, HG00250, HG00264, HG00737, ...",NaN,NaN
2,UW_VH_19141,1,3,177294474,177297489,0.032,3015,133,1,"[HG00180, HG00185, HG00189, HG00237, HG00254, ...",NaN,NaN
3,DEL_pindel_47187,2,18,45379612,45379612,0.550,195,847,66,"[HG00114, HG00132, HG00142, HG00150, HG00272, ...","[HG00108, HG00111, HG00121, HG00125, HG00138, ...",NaN
4,DEL_pindel_24042,2,7,136996507,136996739,0.071,232,184,48,"[HG01241, HG01259, HG01488, HG01890, HG02054, ...","[HG01125, HG01392, HG01403, HG01556, HG01845, ...",NaN


In [6]:
# pad start and end for region queries (start=start-length, stop=stop+length)
df['p_start'] = df.apply(lambda row: row['start'] - row['length'],axis=1)
df['p_stop'] = df.apply(lambda row: row['stop'] + row['length'],axis=1)
df

,id,n_modes,chr,start,stop,allele_freq,length,n_samples,n_ref_samples,mode_1,mode_2,mode_3,p_start,p_stop
0,UW_VH_9038,1,19,54887338,54888354,0.014,1016,51,0,"[HG01125, HG01896, HG01956, HG02009, HG02108, ...",NaN,NaN,54886322,54889370
1,SI_BD_10797,1,10,86801825,86802449,0.033,624,283,139,"[HG00154, HG00159, HG00250, HG00264, HG00737, ...",NaN,NaN,86801201,86803073
2,UW_VH_19141,1,3,177294474,177297489,0.032,3015,133,1,"[HG00180, HG00185, HG00189, HG00237, HG00254, ...",NaN,NaN,177291459,177300504
3,DEL_pindel_47187,2,18,45379612,45379612,0.550,195,847,66,"[HG00114, HG00132, HG00142, HG00150, HG00272, ...","[HG00108, HG00111, HG00121, HG00125, HG00138, ...",NaN,45379417,45379807
4,DEL_pindel_24042,2,7,136996507,136996739,0.071,232,184,48,"[HG01241, HG01259, HG01488, HG01890, HG02054, ...","[HG01125, HG01392, HG01403, HG01556, HG01845, ...",NaN,136996275,136996971
5,BI_GS_DEL1_B2_P0106_507,3,1,105832822,105844373,0.005,11551,24,2,[NA18645],"[NA18574, NA18988]","[HG00446, HG00472, HG00566, HG00704, HG01798, ...",105821271,105855924
6,BI_GS_DEL1_B4_P2674_173,3,19,1950304,1951853,0.155,1549,528,2,"[HG01871, HG01872, HG01873, HG01974, HG02008, ...","[HG00105, HG00146, HG00360, HG00362, HG00593, ...","[HG00140, HG00141, HG00190, HG00276, HG00280, ...",1948755,1953402
7,UW_VH_10394,3,8,96875230,96879582,0.172,4352,580,3,"[HG00099, HG00350, HG01524, HG01761, HG02010, ...","[HG00097, HG00119, HG00122, HG00159, HG00185, ...","[HG00110, HG00111, HG00136, HG00171, HG00178, ...",96870878,96883934
8,BI_GS_DEL1_B2_P0114_484,3,1,113799624,113800089,0.760,465,1893,7,"[HG01605, HG01608, HG01756, HG01757, HG01761, ...","[HG00101, HG00105, HG00106, HG00112, HG00123, ...","[HG00100, HG00108, HG00109, HG00110, HG00111, ...",113799159,113800554


In [7]:
variants=df.id.tolist()
variants

['UW_VH_9038',
 'SI_BD_10797',
 'UW_VH_19141',
 'DEL_pindel_47187',
 'DEL_pindel_24042',
 'BI_GS_DEL1_B2_P0106_507',
 'BI_GS_DEL1_B4_P2674_173',
 'UW_VH_10394',
 'BI_GS_DEL1_B2_P0114_484']

In [8]:
# output dirs 
# variant_name/
# |
# l_ samplot/
# l_ bam/
for v in variants:
    d = os.path.join(dirout,v)
    if not os.path.exists(d):
        os.mkdir(d)
    d_bam = os.path.join(dirout,v,'bam')
    d_samplot = os.path.join(dirout,v,'samplot')
    if not os.path.exists(d_bam):
        os.mkdir(d_bam)
    if not os.path.exists(d_samplot):
        os.mkdir(d_samplot)

In [10]:
# filter index file for mapped reads only
paths = pd.read_csv(idx_1kg_phase3,sep='\t').iloc[:,0]
print(paths.shape)
mask = paths.str.contains(r'\.mapped\.ILLUMINA.*bam$',regex=True)
mapped = paths[mask]
print(mapped.shape)
mapped.head()
mapped_index = '../data/1kg_phase3_mapped.index'
mapped.to_csv('../data/1kg_phase3_mapped.index', header=False,index=False)

(10140,)
(2535,)


In [11]:
pd.read_csv(mapped_index,header=None, usecols=[0]).iloc[:,0]

0       data/HG00096/alignment/HG00096.mapped.ILLUMINA...
1       data/HG00097/alignment/HG00097.mapped.ILLUMINA...
2       data/HG00099/alignment/HG00099.mapped.ILLUMINA...
3       data/HG00100/alignment/HG00100.mapped.ILLUMINA...
4       data/HG00101/alignment/HG00101.mapped.ILLUMINA...
                              ...                        
2530    data/NA21137/alignment/NA21137.mapped.ILLUMINA...
2531    data/NA21141/alignment/NA21141.mapped.ILLUMINA...
2532    data/NA21142/alignment/NA21142.mapped.ILLUMINA...
2533    data/NA21143/alignment/NA21143.mapped.ILLUMINA...
2534    data/NA21144/alignment/NA21144.mapped.ILLUMINA...
Name: 0, Length: 2535, dtype: object

In [249]:
# old
#def download_1kg_bam_region(
#    index_file,
#    sample,
#    chrom,
#    left,
#    right,
#    out,
#    bucket='1000genomes',
#    phase='phase3',
#    tmp='/data/jake/sv-gmm/tmp'
#):
#    print('index_file:',index_file,'sample:',sample,'left:',left,'right:',right,'out:',out)
#    index_series = pd.read_csv(mapped_index,header=None).iloc[:,0]
#    chrom = str(chrom)
#    left = int(left)
#    right = int(right)
#
#    # filter index for sample
#    mask = index_series.str.contains(sample)
#    if bool(mask.sum() > 1):
#        raise ValueError("sample name matched more than one file in index")
#    key_bam = index_series[mask].tolist()[0]
#    print('key_bam:',key_bam)
#    key_bam = os.path.join(phase, key_bam) # e.g., phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
#    key_bai = key_bam + '.bai'
#    # url
#    url_bam = os.path.join(
#        's3://', bucket, key_bam # e.g., s3://1000genomes/phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
#    )
#    # out
#    region_bam=out + '.bam'
#
#    s3.download_file(bucket, key_bai, os.path.join(tmp,key_bai))
#    # download full bam index temporarily, necessary to 
#    # get region bam
#    ### try without tempfile
#    with tempfile.NamedTemporaryFile(delete=True) as temp_file:
#        # s3 dl
#        s3 = boto3.client('s3')
#        s3.download_fileobj(bucket, key_idx, temp_file)
#        print('temp index at:', temp_file.name)
#        # ensure pointer is at beginning of file for pysam reading
#        # (likely not necessary)
#        #temp_file.seek(0)
#        print(url_bam)
#
#        # read server bam with local index (may be able to use remote index, not sure)
#        with pysam.AlignmentFile(url_bam, "rb", index_filename=temp_file.name) as f_in:
#            print('writing region bam at:', region_bam)
#            with pysam.AlignmentFile(region_bam, "wb", header=f_in.header) as f_out:
#                print(chrom,left,right)
#                for read in f_in.fetch(str(chrom), left, right):
#                    f_out.write(read)
#        # index region bam
#        print('indexing', region_bam)
#        pysam.index(region_bam)
#    return None
#download_1kg_bam_region(mapped_index,"HG01125",19,54886338,54889354,'../test')
#        

../test
temp index at: /tmp/tmpmm1ml3to
s3://1000genomes/phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
writing region bam at: ../test.bam
19 54886338 54889354
indexing ../test.bam


In [20]:
#v2
def download_1kg_bam_region(
    index_file,
    sample,
    chrom,
    left,
    right,
    out,
    bucket='1000genomes',
    phase='phase3',
    tmp='/data/jake/sv-gmm/tmp'
):
    print('index_file:',index_file,'sample:',sample,'left:',left,'right:',right,'out:',out)
    index_series = pd.read_csv(mapped_index,header=None).iloc[:,0]
    chrom = str(chrom)
    left = int(left)
    right = int(right)

    # filter index for sample
    mask = index_series.str.contains(sample)
    if bool(mask.sum() > 1):
        raise ValueError("sample name matched more than one file in index")
    key_bam = index_series[mask].tolist()[0]
    print('key_bam:',key_bam)
    key_bam = os.path.join(phase, key_bam) # e.g., phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
    key_bai = key_bam + '.bai'
    # url
    url_bam = os.path.join(
        's3://', bucket, key_bam # e.g., s3://1000genomes/phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
    )
    # out
    region_bam=out + '.bam'
    print('region_bam', region_bam)
    # full idx goes to tmp
    s3 = boto3.client('s3')
    full_idx=os.path.join(tmp,os.path.basename(key_bai))
    s3.download_file(bucket, key_bai, full_idx)

    ## Use pysam's remote access capability
    with pysam.AlignmentFile(url_bam, "rb", index_filename=full_idx) as f_in:
        with pysam.AlignmentFile(region_bam, "wb", header=f_in.header) as f_out:
            for read in f_in.fetch(chrom, left, right):
                f_out.write(read)
    pysam.index(region_bam)
    return None
download_1kg_bam_region(mapped_index,"HG01125",19,54886338,54889354,'test')
        

index_file: ../data/1kg_phase3_mapped.index sample: HG01125 left: 54886338 right: 54889354 out: test
key_bam: data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam
region_bam test.bam


for v in variant: (rows)
    define chr, start, end, 
    for m, s in zip(modes,samples):
            download_1kg_bam(idx, s, chr, start, end, out = s+m)
    for sample in variant:
    

In [21]:
queries = {'sample': [], 'chrom': [], 'p_start':[], 'p_stop':[], 'fname':[]}
for i,row in df.iterrows():
    variant = row['id']
    chrom = row['chr']
    p_start = row['p_start']
    p_stop = row['p_stop']
    data = pd.DataFrame({'sample':[], 'mode':[]})
    for m in ['mode_1', 'mode_2', 'mode_3']:
        if isinstance(row[m], list):
            for s in row[m]:
                data = pd.concat([data, pd.DataFrame({'sample': [s], 'mode': [m]})])
        elif pd.isnull((row[m])):
            print(f'no {m} for {variant}')
        else:
            raise ValueError
    for j, r in data.iterrows():
        sample = r['sample']
        mode = r['mode']
        #queries['variant'].append(variant)
        queries['sample'].append(sample)
        #queries['mode'].append(mode)
        queries['chrom'].append(chrom)
        queries['p_start'].append(p_start)
        queries['p_stop'].append(p_stop)
        queries['fname'].append(os.path.join(dirout,variant,'bam', f'{variant}.{mode}.{sample}.{chrom}.{p_start}.{p_stop}'))
    

no mode_2 for UW_VH_9038
no mode_3 for UW_VH_9038
no mode_2 for SI_BD_10797
no mode_3 for SI_BD_10797
no mode_2 for UW_VH_19141
no mode_3 for UW_VH_19141
no mode_3 for DEL_pindel_47187
no mode_3 for DEL_pindel_24042


In [22]:
df_query = pd.DataFrame(queries)
df_query.insert(0, 'index', mapped_index)
df_query.head()

,index,sample,chrom,p_start,p_stop,fname
0,../data/1kg_phase3_mapped.index,HG01125,19,54886322,54889370,../data/variants/UW_VH_9038/bam/UW_VH_9038.mod...
1,../data/1kg_phase3_mapped.index,HG01896,19,54886322,54889370,../data/variants/UW_VH_9038/bam/UW_VH_9038.mod...
2,../data/1kg_phase3_mapped.index,HG01956,19,54886322,54889370,../data/variants/UW_VH_9038/bam/UW_VH_9038.mod...
3,../data/1kg_phase3_mapped.index,HG02009,19,54886322,54889370,../data/variants/UW_VH_9038/bam/UW_VH_9038.mod...
4,../data/1kg_phase3_mapped.index,HG02108,19,54886322,54889370,../data/variants/UW_VH_9038/bam/UW_VH_9038.mod...


In [23]:
df_query['fname'][0]

'../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG01125.19.54886322.54889370'

In [24]:
# list of tuples for starmap
tuples_list = [row for row in df_query.itertuples(index=False, name=None)]
tuples_list[0:5]

[('../data/1kg_phase3_mapped.index',
  'HG01125',
  19,
  54886322,
  54889370,
  '../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG01125.19.54886322.54889370'),
 ('../data/1kg_phase3_mapped.index',
  'HG01896',
  19,
  54886322,
  54889370,
  '../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG01896.19.54886322.54889370'),
 ('../data/1kg_phase3_mapped.index',
  'HG01956',
  19,
  54886322,
  54889370,
  '../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG01956.19.54886322.54889370'),
 ('../data/1kg_phase3_mapped.index',
  'HG02009',
  19,
  54886322,
  54889370,
  '../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG02009.19.54886322.54889370'),
 ('../data/1kg_phase3_mapped.index',
  'HG02108',
  19,
  54886322,
  54889370,
  '../data/variants/UW_VH_9038/bam/UW_VH_9038.mode_1.HG02108.19.54886322.54889370')]

In [28]:
# single process
#for t in tuples_list:
        #download_1kg_bam_region(*t)
# parallel
with Pool(processes=cpus) as pool:
        results = pool.starmap(download_1kg_bam_region, tuples_list)

index_file:index_file:index_file:index_file:index_file:index_file:index_file:index_file:index_file:index_file:index_file: index_file:index_file: index_file:index_file: index_file: index_file:  index_file:index_file:index_file:index_file:index_file: index_file:index_file:index_file:index_file:   ../data/1kg_phase3_mapped.indexindex_file:index_file:  index_file: index_file:../data/1kg_phase3_mapped.index index_file: index_file:index_file:../data/1kg_phase3_mapped.indexindex_file: index_file:../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index   ../data/1kg_phase3_mapped.index       ../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index ../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index../data/1kg_phase3_mapped.index ../data/1kg_phase3_mapped.index ../data/1kg_phase3_mapped.index     ../data/1kg_phase3_mapped.index  ../data/1kg_phase3_mapped.index   ../data/1kg_phase3_mapped.index../data/1kg_phase3

In [ ]:
padding=500

In [28]:
# single example
#import boto3
#import pysam
#import os
#
#base = 'test'
#bucket='1000genomes'
#key_idx='phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam.bai'
#out_idx=os.path.join(scratch, f'{base}.full.bam.bai')
##url_idx="s3://1000genomes/phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam.bai"
#url_bam="s3://1000genomes/phase3/data/HG01125/alignment/HG01125.mapped.ILLUMINA.bwa.CLM.low_coverage.20120522.bam"
#chrom = "19"
#region_l = 54886338
#region_r= 54889354
#out_bam=os.path.join(scratch, f'{base}.bam')
#
## download index
#s3 = boto3.client('s3')
#s3.download_file(bucket, key_idx, out_idx)
#
### Use pysam's remote access capability
##url = f"s3://{bucket_name}.s3.amazonaws.com/{bam_key}"
#with pysam.AlignmentFile(url_bam, "rb", index_filename=out_idx) as f_in:
#    with pysam.AlignmentFile(out_bam, "wb", header=f_in.header) as f_out:
#        for read in f_in.fetch(chrom, region_l, region_r):
#            f_out.write(read)
## index
#pysam.index(out_bam)
#

''